## GRV API

- acc email: maxharper26@icloud.com
- acc password: kowGot-sonbyk-4fazko
- api key: 4157732f-fa7f-4459-ab26-089e4deab17a
- documentation: https://topaz.grv.org.au/docs/#/codes/GetTrack

In [1]:
import pandas as pd
from topaz import TopazAPI
import time
import requests
from requests.exceptions import HTTPError
from json import JSONDecodeError
import numpy as np

In [2]:
key = '4157732f-fa7f-4459-ab26-089e4deab17a'
key_m = '6a8ffdc7-ee7a-4795-9b98-607e3e838342'
key_p ='34b33827-3ca1-4c2b-a4c5-40fde1d99a84'


In [3]:
topaz_api = TopazAPI(key)

In [4]:
authority_codes = [ 'NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA','NT']
all_races = []
missed_run=False

for code in authority_codes:
    try:
        races = topaz_api.get_races(from_date='2024-03-15', to_date='2024-03-15', owning_authority_code=code)
        all_races.append(races)
    except Exception as e:
        print(f"Error fetching races for {code}: {e}")

all_races_df = pd.concat(all_races, ignore_index=True)
race_ids = list(all_races_df['raceId'].unique())
race_ids

Error fetching races for TAS: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2024-03-15&to=2024-03-15&owningauthoritycode=TAS
Error fetching races for NT: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2024-03-15&to=2024-03-15&owningauthoritycode=NT


[1006656807,
 1006656808,
 1006656809,
 1006656810,
 1006656811,
 1006656812,
 1006656813,
 1006656814,
 1006656815,
 1006656816,
 1006656817,
 1006656818,
 1006656834,
 1006656835,
 1006656837,
 1006656839,
 1006656840,
 1006656842,
 1006656843,
 1006656844,
 1006656845,
 1006656846,
 1006693869,
 1006693870,
 1006693871,
 1006693872,
 1006693873,
 1006693874,
 1006693875,
 1006693876,
 1006693877,
 1006693878,
 1006693879,
 1007029069,
 1007029070,
 1007029071,
 1007029072,
 1007029073,
 1007029074,
 1007029075,
 1007029076,
 1007029077,
 1007029078,
 1007029079,
 1006658811,
 1006658812,
 1006658813,
 1006658814,
 1006658815,
 1006658816,
 1006658817,
 1006658818,
 1006658819,
 1006658820,
 1006658821,
 1007029080,
 1007029081,
 1007029082,
 1007029083,
 1007029084,
 1007029085,
 1007029086,
 1007029087,
 1007029088,
 1007029089,
 1007029090,
 1007029091,
 1006693912,
 1006693913,
 1006693914,
 1006693915,
 1006693916,
 1006693917,
 1006693918,
 1006693919,
 1006693920,
 1006693921,

In [5]:
len(race_ids)

136

In [6]:
# race_ids = pd.read_csv('api_files/missing.csv')['missed']
# missed_run=True
# race_ids

In [7]:
big = ''
start = time.time()
first=0
headers={'X-API-Key':key}
missed=[]



for i in range(first,len(race_ids)):
    rID = race_ids[i]
    url = f'https://topaz.grv.org.au/api/race/{rID}/result'
    try:
        race = requests.get(url.format(rID),headers=headers).json()
    except JSONDecodeError as e:
        race={'error':1,'message':'empty json'}
    
    if('error' in race and 'message' in race):
        print(rID,race['message'])
        missed.append(rID)
    else:
        splits = race['splitTimes']
        
        
        summary = pd.DataFrame(race['runs'])

        cols=['track','distance','raceId','weightInKg','meetingDate','dogName','resultTime','bestFinishTrackAndDistance', 'pir','incomingGrade', 'outgoingGrade', 'rating','resultMarginLengths', 'startPaceCode', 'jumpCode',
           'startPrice','boxNumber']
        summary=summary[cols]
        summary['Plc'] = summary['resultTime'].rank(method='min')

        if(splits!=[]):
            splits = pd.DataFrame(splits)
            splits = splits[['dogName','splitTimeMarker','time']]

            markers = max(splits['splitTimeMarker'])
            for marker in range(1,markers+1):
                split2 = splits[splits['splitTimeMarker']==marker]
                split2=split2.rename(columns={'time':'split_'+str(marker)})[['dogName','split_'+str(marker)]]
                summary = pd.merge(summary,split2,on='dogName',how='outer')


        if(type(big)==str):
            big = summary
        else:
            big = pd.concat([big,summary])

            



end = time.time()
taken = end-start
print('time for',len(race_ids),':',round(taken,3),'avg:',round(taken/len(race_ids),2))


1007071127 empty json
1007071122 empty json
1007071131 empty json
1007071132 empty json
1007071130 empty json
1007071123 empty json
1007071124 empty json
1007071125 empty json
1007071126 empty json
1007071141 empty json
1007071135 empty json
1007071140 empty json
1007071145 empty json
1007071134 empty json
1007071143 empty json
1007071144 empty json
1007071142 empty json
1007071136 empty json
1007071137 empty json
1007071138 empty json
1007071139 empty json
1006741471 empty json
1006741472 empty json
1006741475 empty json
1006741467 empty json
1006741468 empty json
1006741464 empty json
1006741474 empty json
1006741473 empty json
1006741470 empty json
1006741469 empty json
1006741465 empty json
1006741466 empty json
1006693892 empty json
1006693893 empty json
1006693894 empty json
1006693895 empty json
1006693896 empty json
1006693897 empty json
1006693898 empty json
1006693899 empty json
1006693900 empty json
1006693901 empty json
time for 136 : 50.593 avg: 0.37


In [8]:
big

,track,distance,raceId,weightInKg,meetingDate,dogName,resultTime,bestFinishTrackAndDistance,pir,incomingGrade,outgoingGrade,rating,resultMarginLengths,startPaceCode,jumpCode,startPrice,boxNumber,Plc,split_1,split_2
0,The Gardens,400,1006656807,NaN,2024-03-15T00:00:00.000Z,FASCINATE GLORY,NaN,23.380,None,Maiden,Maiden,84,None,None,None,1.0,NaN,NaN,NaN,NaN
1,The Gardens,400,1006656807,NaN,2024-03-15T00:00:00.000Z,LIL PATTI,NaN,24.280,None,Maiden,Maiden,76,None,None,None,1.0,NaN,NaN,NaN,NaN
2,The Gardens,400,1006656807,24.1,2024-03-15T00:00:00.000Z,MEKONG RUBY,23.740,23.740,0,Maiden,Maiden,83,8.25L,None,None,31.0,1.0,5.0,NaN,NaN
3,The Gardens,400,1006656807,26.4,2024-03-15T00:00:00.000Z,KICK ON KATE,23.200,23.200,0,7,Maiden,0,1.00L,None,None,2.7,2.0,1.0,NaN,NaN
4,The Gardens,400,1006656807,23.7,2024-03-15T00:00:00.000Z,RETA'S ANGEL,23.560,23.450,0,Maiden,Maiden,100,5.50L,None,None,6.5,3.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,Healesville,300,1007071128,26.3,2024-03-15T00:00:00.000Z,SON OF DUKE,17.876,17.225,2,7,7,86,11.75L,None,Medium,4.6,6.0,8.0,3.9250,NaN
6,Healesville,300,1007071128,33.4,2024-03-15T00:00:00.000Z,BOOK OF CAPERS,17.418,16.982,3,7,7,94,5.00L,None,Medium,7.0,7.0,4.0,3.9425,NaN
7,Healesville,300,1007071128,29.0,2024-03-15T00:00:00.000Z,ZAMBORA'S GIRL,17.095,16.981,4,7,6,95,1.25L,None,Medium,10.0,8.0,1.0,3.9460,NaN
8,Healesville,300,1007071128,NaN,2024-03-15T00:00:00.000Z,FLASH BANG,NaN,17.130,None,7,7,84,None,None,None,21.0,NaN,NaN,NaN,NaN


In [9]:
big1 = big.copy()
big1['Track']=big1['track'].str.lower()
big1['dogName']=big1['dogName'].str.lower()

to_rename={
    'Plc':'Place',
    'dogName':'DogName',
    'boxNumber':'Box',
    'resultTime':'RunTime',
    'weightInKg':'Weight',
    'startPrice':'StartPrice',
    'pir':'PIR',
    'distance':'Distance',
    'raceId':'RaceId',
    'incomingGrade':'RaceGrade',
    'meetingDate':'date',
    'split_1':'Split'
}

big1=big1.rename(columns=to_rename)
big1 = big1[['Place','DogName','Box','Weight','StartPrice','PIR','Split','RunTime','RaceId','Distance','RaceGrade','Track','date']]
big1['TrainerLName']='poo'
big1['TrainerFName']='poo'

big1['date']=big1['date'].astype(str).str.split('T').str[0]
big1['date']=pd.to_datetime(big1['date'])


import os
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
m=0
print(files)
for file in files:
    num = file.split('_')[1].split('.')[0]
    num=int(num)
    if(num>m):
        m=num
print(m)

big1.to_csv('api_files/api_'+str(m+1)+'.csv',index=False)
big1

['api_8.csv', 'api_9.csv', 'api_17.csv', 'api_16.csv', 'api_14.csv', 'api_28.csv', 'api_29.csv', 'api_15.csv', 'api_11.csv', 'api_10.csv', 'api_12.csv', 'api_13.csv', 'api_36.csv', 'api_22.csv', 'api_23.csv', 'api_21.csv', 'api_35.csv', 'api_34.csv', 'api_20.csv', 'api_18.csv', 'api_24.csv', 'api_30.csv', 'api_31.csv', 'api_25.csv', 'api_19.csv', 'api_33.csv', 'api_27.csv', 'api_26.csv', 'api_32.csv', 'api_7.csv', 'api_6.csv', 'api_4.csv', 'api_5.csv', 'api_1.csv', 'api_2.csv', 'api_3.csv']
36


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,fascinate glory,NaN,NaN,1.0,None,NaN,NaN,1006656807,400,Maiden,the gardens,2024-03-15,poo,poo
1,NaN,lil patti,NaN,NaN,1.0,None,NaN,NaN,1006656807,400,Maiden,the gardens,2024-03-15,poo,poo
2,5.0,mekong ruby,1.0,24.1,31.0,0,NaN,23.740,1006656807,400,Maiden,the gardens,2024-03-15,poo,poo
3,1.0,kick on kate,2.0,26.4,2.7,0,NaN,23.200,1006656807,400,7,the gardens,2024-03-15,poo,poo
4,4.0,reta's angel,3.0,23.7,6.5,0,NaN,23.560,1006656807,400,Maiden,the gardens,2024-03-15,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,8.0,son of duke,6.0,26.3,4.6,2,3.9250,17.876,1007071128,300,7,healesville,2024-03-15,poo,poo
6,4.0,book of capers,7.0,33.4,7.0,3,3.9425,17.418,1007071128,300,7,healesville,2024-03-15,poo,poo
7,1.0,zambora's girl,8.0,29.0,10.0,4,3.9460,17.095,1007071128,300,7,healesville,2024-03-15,poo,poo
8,NaN,flash bang,NaN,NaN,21.0,None,NaN,NaN,1007071128,300,7,healesville,2024-03-15,poo,poo


In [10]:
if(missed_run==False):
    missing  = pd.read_csv('api_files/missing.csv')[['missed']]
    missing2 = pd.DataFrame(missed,columns=['missed'])
    missing3 = pd.concat([missing,missing2])
    missing4 = missing3['missed'].unique()
    missing5 = pd.DataFrame(missing4,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
else:
    missing5 = pd.DataFrame(missed,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
    
missing5

,missed
0,990395989
1,990395990
2,990395991
3,990395992
4,990395993
...,...
371,1006693897
372,1006693898
373,1006693899
374,1006693900


In [11]:
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
files.sort()
files
merged = pd.read_csv('api_files/'+files[0])
for file in files[1:]:
    merged=pd.concat([merged,pd.read_csv('api_files/'+file)])

merged=merged.sort_values(by='date')
merged['Track']=np.where(merged['Track'].str.contains('dport @'),'hobart',merged['Track'])
merged

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,6.0,chick magnet,1.0,30.5,5.5,566.0,NaN,19.700,990412069,336,5,muswellbrook,2024-01-28,poo,poo
1331,1.0,monkey's style,7.0,34.5,4.9,111.0,5.1800,24.282,992122290,435,5,sale,2024-01-28,poo,poo
1330,3.0,rumour confirmed,6.0,31.9,38.7,743.0,5.4425,24.537,992122290,435,5,sale,2024-01-28,poo,poo
1329,8.0,crew,5.0,34.5,57.1,678.0,5.4075,25.242,992122290,435,5,sale,2024-01-28,poo,poo
1328,7.0,persuasion,4.0,27.1,20.2,887.0,5.4775,25.181,992122290,435,5,sale,2024-01-28,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2.0,kelly bolt,7.0,30.4,34.0,563.0,5.7600,30.470,1007029069,520,Maiden,wentworth park,2024-03-15,poo,poo
291,4.0,bradshaw,8.0,32.1,9.5,776.0,5.7800,31.180,1007029069,520,Maiden,wentworth park,2024-03-15,poo,poo
292,NaN,steal a march,NaN,NaN,1.0,NaN,NaN,NaN,1007029070,520,6,wentworth park,2024-03-15,poo,poo
294,3.0,computer model,2.0,31.9,3.3,422.0,5.5200,30.370,1007029070,520,7,wentworth park,2024-03-15,poo,poo


In [12]:
a = pd.read_csv('../allRaces2.csv')
b = merged.copy()
c = pd.concat([a,b])
#c = c.drop(columns=['Unnamed: 0.1'])
c = c.drop_duplicates(subset=['DogName','RunTime','date'],keep='first')

c['Track']=np.where(c['Track'].str.contains('murray'),'murray bridge',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])



c.to_csv('../allRaces1.csv',index=False)
c

/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_52491/609129207.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_52491/609129207.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_52491/609129207.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,raine allen,1.0,27.4,2.4,111,6.68,28.66,335811282,500,Restricted Win,bendigo,2018-07-01,Grenfell,C
1,2.0,surf a lot,2.0,32.8,6.3,332,7.01,28.82,335811282,500,Restricted Win,bendigo,2018-07-01,Tyley,C
2,3.0,pingin' bee,6.0,25.5,9.3,443,7.24,28.93,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
3,4.0,lucas the great,7.0,32.2,9.1,655,7.31,29.03,335811282,500,Restricted Win,bendigo,2018-07-01,Hamilton,E
4,5.0,quavo,4.0,28.9,3.4,766,7.38,29.05,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2.0,kelly bolt,7.0,30.4,34.0,563.0,5.76,30.47,1007029069,520,Maiden,wentworth park,2024-03-15,poo,poo
291,4.0,bradshaw,8.0,32.1,9.5,776.0,5.78,31.18,1007029069,520,Maiden,wentworth park,2024-03-15,poo,poo
292,NaN,steal a march,NaN,NaN,1.0,NaN,NaN,NaN,1007029070,520,6,wentworth park,2024-03-15,poo,poo
294,3.0,computer model,2.0,31.9,3.3,422.0,5.52,30.37,1007029070,520,7,wentworth park,2024-03-15,poo,poo


In [13]:
tks = c['Track'].unique()
tks.sort()
tks

array(['addington', 'albion park', 'angle park', 'armidale', 'ascot park',
       'ballarat', 'bathurst', 'bendigo', 'broken hill', 'bulli',
       'bundaberg', 'cairns', 'cambridge', 'canberra', 'cannington',
       'capalaba', 'casino', 'central park', 'clonmel', 'coonabarabran',
       'coonamble', 'cowra', 'cranbourne', 'crayford', 'curraheen park',
       'dapto', 'darwin', 'devonport', 'doncaster', 'drumbo park',
       'dubbo', 'dundalk', 'enniscorthy', 'galway', 'gawler', 'geelong',
       'gosford', 'goulburn', 'grafton', 'gunnedah', 'harlow', 'hatrick',
       'healesville', 'henlow', 'hobart', 'horsham', 'hove', 'ipswich',
       'kempsey', 'kilkenny', 'kinsley', 'launceston', 'lifford',
       'limerick', 'lismore', 'lithgow', 'maitland', 'manawatu',
       'mandurah', 'manukau', 'monmore', 'moree', 'mount gambier',
       'mudgee', 'mullingar', 'murray bridge', 'muswellbrook',
       'newbridge', 'newcastle', 'northam', 'nottingham', 'nowra',
       'oxford', 'pelaw grange

In [14]:
## still missing 
for rID in missing5['missed']:
    url = f'https://topaz.grv.org.au/api/race/{rID}/runs'
    try:
        race = requests.get(url.format(rID),headers=headers).json()[0]
        print(race['track'],race['meetingDate'],race['distance'])
    except JSONDecodeError as e:
        print(rID,'fucked')
        
# race

Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 498
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murr

Ballarat 2024-03-13T00:00:00.000Z 390
Ballarat 2024-03-13T00:00:00.000Z 390
Ballarat 2024-03-13T00:00:00.000Z 450
Ballarat 2024-03-13T00:00:00.000Z 450
Ballarat 2024-03-13T00:00:00.000Z 450
Ballarat 2024-03-13T00:00:00.000Z 450
Ballarat 2024-03-13T00:00:00.000Z 450
Ballarat 2024-03-13T00:00:00.000Z 390
Ballarat 2024-03-13T00:00:00.000Z 390
Mandurah 2024-03-14T00:00:00.000Z 400
Mandurah 2024-03-14T00:00:00.000Z 400
Mandurah 2024-03-14T00:00:00.000Z 300
Mandurah 2024-03-14T00:00:00.000Z 300
Mandurah 2024-03-14T00:00:00.000Z 488
Mandurah 2024-03-14T00:00:00.000Z 300
Mandurah 2024-03-14T00:00:00.000Z 400
Mandurah 2024-03-14T00:00:00.000Z 300
Mandurah 2024-03-14T00:00:00.000Z 300
Healesville 2024-03-15T00:00:00.000Z 300
Healesville 2024-03-15T00:00:00.000Z 350
Healesville 2024-03-15T00:00:00.000Z 350
Healesville 2024-03-15T00:00:00.000Z 350
Healesville 2024-03-15T00:00:00.000Z 350
Healesville 2024-03-15T00:00:00.000Z 300
Healesville 2024-03-15T00:00:00.000Z 300
Healesville 2024-03-15T00:00:

In [16]:
big1

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,spaghetti arms,NaN,NaN,12.0,None,NaN,NaN,1007040570,400,Maiden,geelong,2024-03-15,poo,poo
1,2.0,blackpool ranger,1.0,32.8,4.0,22,8.56000,23.280,1007040570,400,Maiden,geelong,2024-03-15,poo,poo
2,6.0,spinning stars,2.0,24.4,18.0,56,8.92750,24.034,1007040570,400,Maiden,geelong,2024-03-15,poo,poo
3,7.0,rose of terang,3.0,23.5,41.0,77,9.20925,24.632,1007040570,400,Maiden,geelong,2024-03-15,poo,poo
4,5.0,platypus yaala,4.0,31.2,7.5,65,8.92925,23.851,1007040570,400,Maiden,geelong,2024-03-15,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2.0,penny blue demon,3.0,28.9,10.0,11,8.42000,22.976,1007040569,400,5,geelong,2024-03-15,poo,poo
4,1.0,smokin' arrow,4.0,32.0,3.5,22,8.59500,22.931,1007040569,400,5,geelong,2024-03-15,poo,poo
5,5.0,solar toby,5.0,32.0,16.0,54,8.73675,23.507,1007040569,400,5,geelong,2024-03-15,poo,poo
6,3.0,ravenna bale,6.0,29.1,7.5,33,8.59675,23.090,1007040569,400,5,geelong,2024-03-15,poo,poo
